**Importing the Dependencies**

In [53]:
import numpy as np
import pandas as pd
import re #to search a text in a document
from nltk.corpus import stopwords #corpus means body of a particular text, nltk stands for natural language toolkit, stopwards does not value that much in a document, like the articles and the auxiliary verbs
from nltk.stem.porter import PorterStemmer #Stemmer takes a word and removes it's prefix and suffix and returns the root word of it.
from sklearn.feature_extraction.text import TfidfVectorizer #text into feature vectors transformation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**Download stopwards from nltk library**

In [54]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
print(stopwords.words('english')) #checking the stopwords

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Preprocessing**

In [56]:
news_dataset = pd.read_csv('/content/train.csv')
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [57]:
news_dataset.shape

(20800, 5)

In [58]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [59]:
#replacing the null values with empty strings
news_dataset = news_dataset.fillna('')

In [60]:
#mergining the title and author columns
news_dataset['Content'] = news_dataset['author']+''+news_dataset['title']
print(news_dataset['Content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: Content, Length: 20800, dtype: object


In [61]:
#separating the data and labels
X = news_dataset.drop(columns = 'label', axis = 1)
Y = news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

**Stemming Procedures**

Stemming is the process of reducing a word to it's root word.

Example: actor, actress, acting. For all this word, the root word is act.

In [62]:
port_stem = PorterStemmer()

In [63]:
def stemming(content): #creatig a function called stemming. Content is not fixed. We can use random words in the parenthesis
  stemmed_content = re.sub('[^a-zA-z]',' ', content) #calling the regular expression library by 're', useful for searching a paragraph or text. Sub means subtitutes certain values. Upper arrows means exclusion. In the set 'a-zA-Z', as we need only alphabtes, not commans or numbers, that included here. So, that particular lines basically removes everything that is not alphabate. and it feeds on 'content' column. if there is number or comma, that will be replaced with a space. so, a space was given.
  stemmed_content = stemmed_content.lower() #converting all latters into lower case latters.
  stemmed_content = stemmed_content.split() #splitting it to respective list. All this words and text will be converted to a list.
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stemming all the words but removing the stopwords. মানে হল, word stemmed_content এ ঢুকাবো, stopwords হলে ঢুকাবো না।
  stemmed_content = ' '.join(stemmed_content) #joint all the words which was process in this cell.
  return stemmed_content

In [64]:
news_dataset['Content'] = news_dataset['Content'].apply(stemming) #the content column will be precessed in the stemming function and return in a new column content

In [65]:
print(news_dataset['Content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: Content, Length: 20800, dtype: object


In [66]:
#separating the data and label
X = news_dataset['Content'].values #habijabi removed as Id
Y = news_dataset['label'].values
print(X)
print(Y)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']
[1 0 1 ... 0 1 1]


In [67]:
#converting text to neumerical data
vectorizer = TfidfVectorizer() #giving a number to the repeated words to lessen to significance. As, in the movie review 'Joker', maintaing the movie title again and again has less significance.
vectorizer.fit(X)

X = vectorizer.transform(X) #X is transformed inthe Tfidfvector

In [68]:
print(X)

  (0, 26322)	0.28088379401596425
  (0, 22702)	0.2552336018069161
  (0, 14997)	0.43006226759639316
  (0, 14533)	0.29177259684200296
  (0, 12760)	0.24619727512767195
  (0, 8002)	0.23133661742488731
  (0, 6250)	0.2839932825877813
  (0, 5946)	0.35488202138141456
  (0, 4983)	0.2472595823572816
  (0, 4187)	0.3625320323150658
  (0, 552)	0.2694167078545385
  (1, 27949)	0.36911845953845024
  (1, 11292)	0.24166773097712638
  (1, 8753)	0.5258635625386451
  (1, 5893)	0.31810058109638056
  (1, 4744)	0.23338756776626793
  (1, 3833)	0.45980466668763476
  (1, 3255)	0.18652439327549428
  (1, 2596)	0.3562953366945267
  (2, 26217)	0.3665032495181434
  (2, 16336)	0.43295215406038445
  (2, 9435)	0.30743020569262086
  (2, 8548)	0.3411947414020896
  (2, 5217)	0.40440534260277944
  (2, 5098)	0.5511414848555652
  :	:
  (20797, 25758)	0.08220218573989037
  (20797, 25299)	0.3119640221826561
  (20797, 22063)	0.24902354987792552
  (20797, 20755)	0.2729578683228216
  (20797, 20470)	0.249994989010826
  (20797, 17481

**Train Test Split**

In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, stratify = Y, random_state = 3)

In [71]:
model = LogisticRegression()

In [72]:
model.fit(X_train, Y_train)

LogisticRegression()

**Evaluation**

In [73]:
#accuracy_score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print("Accuracy score of the Training Data: ", training_data_accuracy)

Accuracy score of the Training Data:  0.9731837606837607


In [74]:
#accuracy_score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print("Accuracy score of the Test Data: ", test_data_accuracy)

Accuracy score of the Test Data:  0.9519230769230769


**Making a predictive System**

In [80]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [81]:
print(Y_test[5])

1
